In [1]:
from datasets import load_dataset, Dataset
import pandas as pd
from tqdm import  tqdm

/Users/dominik/venvs/master_thesis/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/dominik/venvs/master_thesis/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<h1> OASST <h1>

In [ ]:
oass_train = load_dataset("OpenAssistant/oasst2", split="train").to_pandas()
oass_valid = load_dataset("OpenAssistant/oasst2", split="validation").to_pandas()
oass_full = pd.concat([oass_train, oass_valid,])
oass_full.reset_index(drop=True, inplace=True)

In [ ]:
oass_full["review_result"].value_counts()

In [72]:
oass_train = load_dataset("OpenAssistant/oasst2", split="train").to_pandas()
oass_valid = load_dataset("OpenAssistant/oasst2", split="validation").to_pandas()
oass_full = pd.concat([oass_train, oass_valid,])
oass_full.reset_index(drop=True, inplace=True)

needed_langs = ["en", "ar", "de", "es", "vi", "zh"]
rows = []
for lang in tqdm(needed_langs):
    print(f"Processing lang: {lang}")
    filtered_df = oass_full[(oass_full["lang"] == lang) & (oass_full["role"] == "assistant")]
    for i, answer in filtered_df.iterrows():
        query = oass_full[oass_full["message_id"] == answer["parent_id"]]["text"].iloc[0]
        rows.append([answer["lang"], answer["message_id"], answer["parent_id"], answer["user_id"], answer["created_date"], query, answer["text"], answer["review_count"]])
        
filtered_dataset = pd.DataFrame(rows, columns=["lang", "message_id", "parent_id", "user_id", "created_date", "query", "answer", "review_count"])
filtered_dataset.drop_duplicates(subset="answer", inplace=True)
filtered_dataset.reset_index(drop=True, inplace=True)

  0%|          | 0/6 [00:00<?, ?it/s]

Processing lang: en


 17%|█▋        | 1/6 [02:16<11:23, 136.67s/it]

Processing lang: ar


 33%|███▎      | 2/6 [02:16<03:45, 56.40s/it] 

...

 67%|██████▋   | 4/6 [03:33<01:34, 47.47s/it]

Processing lang: vi


 83%|████████▎ | 5/6 [03:34<00:30, 30.53s/it]

Processing lang: zh


100%|██████████| 6/6 [03:52<00:00, 38.76s/it]


In [73]:
hf_dataset = Dataset.from_pandas(filtered_dataset)
hf_dataset.push_to_hub("dkoterwa/oasst2_filtered_retrieval")

<h1> MKQA <h1>

In [2]:
mkqa = load_dataset("mkqa", split="train").to_pandas()

In [24]:
needed_langs = ["en", "ar", "de", "es", "vi", "zh_cn"]
rows = []
for i, row in tqdm(mkqa.iterrows(), total=mkqa.shape[0]):
    for lang in needed_langs:
        rows.append([lang, row["example_id"], row["queries"][lang], row["answers"][lang][0]["text"]])
        
filtered_dataset = pd.DataFrame(rows, columns=["lang", "example_id", "query", "answer"])
filtered_dataset.dropna(inplace=True)
filtered_dataset.reset_index(drop=True, inplace=True)

100%|██████████| 10000/10000 [00:00<00:00, 30089.21it/s]


In [28]:
hf_dataset = Dataset.from_pandas(filtered_dataset)
hf_dataset.push_to_hub("dkoterwa/mkqa_filtered")

Uploading the dataset shards: 100%|██████████| 1/1 [00:03<00:00,  3.72s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/dkoterwa/mkqa_filtered/commit/acdcc09e2035ce21e403f81303826a9d0a6697be', commit_message='Upload dataset', commit_description='', oid='acdcc09e2035ce21e403f81303826a9d0a6697be', pr_url=None, pr_revision=None, pr_num=None)